## Converting to protobuf for electric system

In [8]:
import os
import random
import importlib.util

import numpy as np
from feems.components_model.component_electric import (
    Battery,
    ElectricComponent,
    ElectricMachine,
    FuelCell,
    SerialSystemElectric,
    Genset,
    FuelCellSystem,
    BatterySystem,
    COGES,
)
from feems.components_model.component_mechanical import COGAS, Engine
from feems.fuel import FuelOrigin, TypeFuel
from feems.system_model import (
    ElectricPowerSystem,
    MechanicalPropulsionSystem,
    MechanicalPropulsionSystemWithElectricPowerSystem,
)
from feems.types_for_feems import NOxCalculationMethod, TypePower, TypeComponent
from MachSysS.convert_to_protobuf import (
    convert_electric_system_to_protobuf,
    convert_electric_system_to_protobuf_machinery_system,
    convert_mechanical_propulsion_system_with_electric_system_to_protobuf,
    convert_mechanical_system_to_protobuf,
)

# Dynamic import to resolve 'tests' package conflict
# machinery-system-structure/tests conflicts with feems/tests when both are in path
# Loading utility module directly from file path bypasses the package resolution issue
file_path = os.path.join(os.getcwd(), "tests", "utility.py")
if not os.path.exists(file_path): 
    # Fallback if cwd is incorrect
    file_path = os.path.join(os.path.dirname(os.path.abspath("__file__")) if "__file__" in locals() else os.getcwd(), "tests", "utility.py")
    
spec = importlib.util.spec_from_file_location("local_tests_utility", file_path)
local_utility = importlib.util.module_from_spec(spec)
spec.loader.exec_module(local_utility)

create_shaftline_with_components = local_utility.create_shaftline_with_components
create_switchboard_with_components = local_utility.create_switchboard_with_components

components = []
for switchboard_id in [1, 2, 3]:
    components.extend(
        create_switchboard_with_components(
            switchboard_id=switchboard_id,
            rated_power_available_total=10000 * random.random(),
            no_power_sources=2,
            no_power_consumer=3,
        ).components
    )
electric_system = ElectricPowerSystem(
    name="El 1", power_plant_components=components, bus_tie_connections=[(1, 2), (2, 3)]
)
electric_system_proto = convert_electric_system_to_protobuf(electric_system)
assert len(electric_system.switchboards) == len(electric_system_proto.switchboards)

mechanical_system = MechanicalPropulsionSystem(
    name="Me 1",
    components_list=create_shaftline_with_components(
        shaft_line_id=1,
        rated_power_available_total=10000,
        no_power_consumer=1,
        no_power_sources=1,
    ).components,
)
mechanical_system_proto = convert_mechanical_system_to_protobuf(mechanical_system)
assert len(mechanical_system_proto.shaft_lines) == len(mechanical_system.shaft_line)
system_feems = MechanicalPropulsionSystemWithElectricPowerSystem(
    name="ME Propulsion System",
    mechanical_system=mechanical_system,
    electric_system=electric_system,
)
system_proto = convert_mechanical_propulsion_system_with_electric_system_to_protobuf(system_feems)

# Create an electric propulsion system
number_gensets = 4
gensets = []
for index in range(number_gensets):
    gensets.append(
        Genset(
            name=f"Genset {index + 1}",
            aux_engine=Engine(
                type_=TypeComponent.AUXILIARY_ENGINE,
                rated_power=1000,
                rated_speed=1000,
                bsfc_curve=np.array(
                    [[0.1, 270], [0.25, 240], [0.5, 220], [0.75, 210], [1.0, 220]]
                ),
                nox_calculation_method=NOxCalculationMethod.TIER_3,
                fuel_type=TypeFuel.DIESEL,
                fuel_origin=FuelOrigin.FOSSIL,
            ),
            generator=ElectricMachine(
                type_=TypeComponent.GENERATOR,
                power_type=TypePower.POWER_SOURCE,
                name=f"Generator {index + 1}",
                rated_power=1000,
                rated_speed=1000,
                eff_curve=np.array(
                    [
                        [0.1, 0.70],
                        [0.25, 0.85],
                        [0.5, 0.96],
                        [0.75, 0.97],
                    ]
                ),
                switchboard_id=int(index / 2) + 1,
            ),
        )
    )
energy_storages = []
power_consumers = []
fuel_cell_systems = []
coges = []
number_batteries = 2
for index in range(number_batteries):
    energy_storages.append(
        BatterySystem(
            name="Battery system 1",
            battery=Battery(
                name="Battery 1",
                rated_capacity_kwh=1000,
                charging_rate_c=1,
                discharge_rate_c=1,
                switchboard_id=index + 1,
            ),
            converter=ElectricComponent(
                type_=TypeComponent.POWER_CONVERTER,
                power_type=TypePower.POWER_TRANSMISSION,
                name="Converter 1",
                rated_power=1000,
                eff_curve=np.array(
                    [[0.1, 0.90], [0.25, 0.95], [0.5, 0.96], [0.75, 0.97], [1.0, 0.96]]
                ),
            ),
            switchboard_id=index + 1,
        )
    )

number_propulsion_drives = 2
for index in range(number_propulsion_drives):
    power_consumers.append(
        SerialSystemElectric(
            name="Propulsion Drive 1",
            type_=TypeComponent.PROPULSION_DRIVE,
            power_type=TypePower.POWER_CONSUMER,
            components=[
                ElectricComponent(
                    type_=TypeComponent.POWER_CONVERTER,
                    power_type=TypePower.POWER_TRANSMISSION,
                    name="Converter 1",
                    rated_power=2000,
                    eff_curve=np.array(
                        [
                            [0.1, 0.90],
                            [0.25, 0.95],
                            [0.5, 0.96],
                            [0.75, 0.97],
                            [1.0, 0.96],
                        ]
                    ),
                    switchboard_id=index + 1,
                ),
                ElectricMachine(
                    type_=TypeComponent.SYNCHRONOUS_MACHINE,
                    power_type=TypePower.POWER_CONSUMER,
                    name="Propulsion motor 1",
                    rated_power=2000,
                    rated_speed=1000,
                    eff_curve=np.array(
                        [
                            [0.1, 0.70],
                            [0.25, 0.85],
                            [0.5, 0.96],
                            [0.75, 0.97],
                            [1.0, 0.96],
                        ]
                    ),
                    switchboard_id=index + 1,
                ),
            ],
            switchboard_id=index + 1,
            rated_power=2000,
            rated_speed=1000,
        )
    )

# Add other load
power_consumers.append(
    ElectricComponent(
        type_=TypeComponent.OTHER_LOAD,
        power_type=TypePower.POWER_CONSUMER,
        name="Other load 1",
        rated_power=1000,
        switchboard_id=1,
    )
)

# Add fuel cells
number_fuel_cells = 2
for index in range(number_fuel_cells):
    fuel_cell_systems.append(
        FuelCellSystem(
            name=f"Fuel cell system {index + 1}",
            fuel_cell_module=FuelCell(
                name=f"Fuel cell {index + 1}",
                rated_power=1000,
                eff_curve=np.array(
                    [[0.1, 0.70], [0.25, 0.85], [0.5, 0.96], [0.75, 0.97], [1.0, 0.96]]
                ),
                fuel_type=TypeFuel.HYDROGEN,
                fuel_origin=FuelOrigin.RENEWABLE_NON_BIO,
            ),
            converter=ElectricComponent(
                type_=TypeComponent.POWER_CONVERTER,
                power_type=TypePower.POWER_TRANSMISSION,
                name="Converter 1",
                rated_power=3150,
                eff_curve=np.array(
                    [[0.1, 0.90], [0.25, 0.95], [0.5, 0.96], [0.75, 0.97], [1.0, 0.96]]
                ),
            ),
            switchboard_id=index + 1,
            number_modules=3,
        )
    )

# Create the electric system
electric_system = ElectricPowerSystem(
    name="Electric propulsion system",
    power_plant_components=gensets + energy_storages + power_consumers + fuel_cell_systems,
    bus_tie_connections=[(1, 2)],
)

system_proto = convert_electric_system_to_protobuf_machinery_system(electric_system)
fuel_cells = [
    subsystem.fuel_cell
    for switchboard in system_proto.electric_system.switchboards
    for subsystem in switchboard.subsystems
    if subsystem.HasField("fuel_cell")
]
for each_fuel_cell in fuel_cells:
    assert each_fuel_cell.number_modules == 3

with open(os.path.join("tests", "system_proto.mss"), "wb") as file:
    file.write(system_proto.SerializeToString())

# Create the electric system with COGES
number_coges = 2
rated_power = 10000
gas_turbine_power_curve = np.array([[0.1, 0.2], [0.25, 0.4], [0.5, 0.6], [0.75, 0.7], [1.0, 0.75]])
gas_turbine_power_curve[:, 1] *= rated_power * gas_turbine_power_curve[:, 0]
steam_turbine_power_curve = gas_turbine_power_curve.copy()
steam_turbine_power_curve[:, 1] = (
    rated_power * steam_turbine_power_curve[:, 0] - gas_turbine_power_curve[:, 1]
)
for i in range(number_coges):
    coges.append(
        COGES(
            name=f"COGES {i + 1}",
            cogas=COGAS(
                name=f"COGAS {i + 1}",
                rated_power=10000,
                rated_speed=1000,
                eff_curve=np.array([0.9]),
                gas_turbine_power_curve=gas_turbine_power_curve,
                steam_turbine_power_curve=steam_turbine_power_curve,
                fuel_type=TypeFuel.NATURAL_GAS,
                fuel_origin=FuelOrigin.FOSSIL,
                nox_calculation_method=NOxCalculationMethod.TIER_3,
            ),
            generator=ElectricMachine(
                name=f"Generator {i + 1}",
                rated_power=10000,
                rated_speed=1000,
                power_type=TypePower.POWER_SOURCE,
                type_=TypeComponent.GENERATOR,
                eff_curve=np.array([[0.1, 0.70], [0.25, 0.85], [0.5, 0.96], [0.75, 0.97]]),
                switchboard_id=i + 1,
            ),
        )
    )

electric_system_with_coges = ElectricPowerSystem(
    name="Electric propulsion system with coges",
    power_plant_components=energy_storages + power_consumers + coges + fuel_cell_systems,
    bus_tie_connections=[(1, 2)],
)

system_proto_with_coges = convert_electric_system_to_protobuf_machinery_system(
    electric_system_with_coges
)
with open(os.path.join("tests", "system_proto_with_coges.mss"), "wb") as file:
    file.write(system_proto_with_coges.SerializeToString())

print(system_proto_with_coges)

2026-01-29 17:36:19,582 - local_tests_utility - WARNING - Efficiency of generator is not supplied, using random monotonic curve
2026-01-29 17:36:19,583 - local_tests_utility - WARNING - Efficiency of engine is not supplied, using random monotonic curve


name: "Electric propulsion system with coges"
propulsion_type: ELECTRIC
maximum_allowed_genset_load_percentage: 80
electric_system {
  switchboards {
    switchboard_id: 1
    subsystems {
      converter1 {
        name: "Converter 1"
        rated_power_kw: 1000
        efficiency {
          curve {
            x_label: "power load"
            y_label: "efficiency"
            curve {
              points {
                x: 0.1
                y: 0.9
              }
              points {
                x: 0.25
                y: 0.95
              }
              points {
                x: 0.5
                y: 0.96
              }
              points {
                x: 0.75
                y: 0.97
              }
              points {
                x: 1
                y: 0.96
              }
            }
          }
        }
        order_from_switchboard_or_shaftline: 1
        uid: "b6e7516e-eacc-4e5b-a16f-ce548c0753a7"
      }
      battery {
        name: "Batte

## Test converting to protobuf for mechanical system

In [16]:
# Create a system
import os
from functools import reduce

import numpy as np
from feems.components_model.component_electric import (
    ElectricComponent,
    ElectricMachine,
    Genset,
    PTIPTO,
)
from feems.components_model.component_mechanical import (
    Engine,
    EngineDualFuel,
    MainEngineForMechanicalPropulsion,
    MechanicalPropulsionComponent,
)
from feems.fuel import FuelOrigin, TypeFuel
from feems.system_model import (
    ElectricPowerSystem,
    MechanicalPropulsionSystem,
    MechanicalPropulsionSystemWithElectricPowerSystem,
    HybridPropulsionSystem,
)
from feems.types_for_feems import Power_kW, Speed_rpm, TypeComponent, TypePower
from MachSysS.convert_to_protobuf import (
    convert_mechanical_propulsion_system_with_electric_system_to_protobuf,
    convert_hybrid_propulsion_system_to_protobuf
)

# Create the engine
bsfc_for_main_engine = np.array(
    [
        [0.25, 151.2],
        [0.30, 149.5],
        [0.40, 146.3],
        [0.50, 143.2],
        [0.60, 141.7],
        [0.70, 141.0],
        [0.75, 140.8],
        [0.80, 140.9],
        [0.85, 141.0],
        [0.90, 141.4],
        [0.95, 141.9],
        [1.00, 142.8],
    ]
)
bspfc_for_main_engine = np.array(
    [
        [0.25, 1.6],
        [0.30, 1.4],
        [0.40, 1.1],
        [0.50, 0.9],
        [0.60, 0.8],
        [0.70, 0.8],
        [0.75, 0.7],
        [0.80, 0.7],
        [0.85, 0.7],
        [0.90, 0.6],
        [0.95, 0.6],
        [1.00, 0.6],
    ]
)
main_engine = MainEngineForMechanicalPropulsion(
    engine=EngineDualFuel(
        type_=TypeComponent.MAIN_ENGINE,
        name="Main engine",
        rated_power=Power_kW(18000),
        rated_speed=Speed_rpm(60),
        bsfc_curve=bsfc_for_main_engine,
        fuel_type=TypeFuel.NATURAL_GAS,
        fuel_origin=FuelOrigin.FOSSIL,
        bspfc_curve=bspfc_for_main_engine,
        pilot_fuel_type=TypeFuel.DIESEL,
        nox_calculation_method=NOxCalculationMethod.TIER_3,
    ),
    name="Main engine for mechanical propulsion",
    shaft_line_id=1,
)
propeller = MechanicalPropulsionComponent(
    type_=TypeComponent.PROPELLER_LOAD,
    power_type=TypePower.POWER_CONSUMER,
    name="Propeller",
    rated_power=Power_kW(25000),
    rated_speed=Speed_rpm(60),
    shaft_line_id=1,
)
mechanical_propulsion_system = MechanicalPropulsionSystem(
    name="Mechanical propulsion system", components_list=[main_engine, propeller]
)
number_gensets = 3
bsfc_for_aux_engine = np.array(
    [
        [0.25, 248.0],
        [0.30, 235.0],
        [0.35, 225.0],
        [0.40, 218.0],
        [0.45, 213.0],
        [0.50, 208.0],
        [0.55, 205.0],
        [0.60, 203.0],
        [0.65, 200.0],
        [0.70, 199.0],
        [0.75, 197.0],
        [0.80, 197.0],
        [0.85, 196.0],
        [0.90, 196.0],
        [0.95, 196.0],
        [1.00, 195.2],
    ]
)
gensets = [
    Genset(
        name=f"Genset {i + 1}",
        aux_engine=Engine(
            type_=TypeComponent.AUXILIARY_ENGINE,
            name=f"Aux engine {i + 1}",
            rated_power=Power_kW(1290),
            rated_speed=Speed_rpm(900),
            bsfc_curve=bsfc_for_aux_engine,
            fuel_type=TypeFuel.NATURAL_GAS,
            fuel_origin=FuelOrigin.FOSSIL,
            nox_calculation_method=NOxCalculationMethod.TIER_3,
        ),
        generator=ElectricMachine(
            type_=TypeComponent.GENERATOR,
            name=f"Generator {i + 1}",
            rated_power=Power_kW(1000),
            rated_speed=Speed_rpm(900),
            eff_curve=np.array(
                [
                    [0.0, 0.88],
                    [0.1, 0.89],
                    [0.2, 0.90],
                    [0.3, 0.91],
                    [0.4, 0.92],
                    [0.5, 0.93],
                    [0.6, 0.93],
                    [0.7, 0.94],
                    [0.8, 0.94],
                    [0.9, 0.95],
                    [1.0, 0.95],
                ]
            ),
            power_type=TypePower.POWER_SOURCE,
            switchboard_id=int(i / 2) + 1,
        ),
    )
    for i in range(number_gensets)
]
other_load = ElectricComponent(
    type_=TypeComponent.OTHER_LOAD,
    name="Other load",
    rated_power=Power_kW(4000),
    power_type=TypePower.POWER_CONSUMER,
    switchboard_id=1,
)
electric_power_system = ElectricPowerSystem(
    name="Electric power system",
    power_plant_components=[*gensets, other_load],
    bus_tie_connections=[(1, 1)],
)
machinery_system = MechanicalPropulsionSystemWithElectricPowerSystem(
    name="Base case system",
    mechanical_system=mechanical_propulsion_system,
    electric_system=electric_power_system,
)

In [17]:
# Conversion of the mechanical system
mechanical_system_proto = convert_mechanical_system_to_protobuf(
    mechanical_propulsion_system=mechanical_propulsion_system
)
print(mechanical_system_proto)

shaft_lines {
  shaft_line_id: 1
  subsystems {
    engine {
      name: "Main engine"
      rated_power_kw: 18000
      rated_speed_rpm: 60
      bsfc {
        curve {
          x_label: "power load"
          y_label: "bsfc"
          curve {
            points {
              x: 0.25
              y: 151.2
            }
            points {
              x: 0.3
              y: 149.5
            }
            points {
              x: 0.4
              y: 146.3
            }
            points {
              x: 0.5
              y: 143.2
            }
            points {
              x: 0.6
              y: 141.7
            }
            points {
              x: 0.7
              y: 141
            }
            points {
              x: 0.75
              y: 140.8
            }
            points {
              x: 0.8
              y: 140.9
            }
            points {
              x: 0.85
              y: 141
            }
            points {
              x: 0.9
  

In [18]:
# Conversion of the mechanical propulsion system with electric power system
mechanical_propulsion_system_with_electric_system_proto = (
    convert_mechanical_propulsion_system_with_electric_system_to_protobuf(
        system_feems=machinery_system
    )
)
print(mechanical_propulsion_system_with_electric_system_proto)

name: "Base case system"
maximum_allowed_genset_load_percentage: 80
mechanical_system {
  shaft_lines {
    shaft_line_id: 1
    subsystems {
      engine {
        name: "Main engine"
        rated_power_kw: 18000
        rated_speed_rpm: 60
        bsfc {
          curve {
            x_label: "power load"
            y_label: "bsfc"
            curve {
              points {
                x: 0.25
                y: 151.2
              }
              points {
                x: 0.3
                y: 149.5
              }
              points {
                x: 0.4
                y: 146.3
              }
              points {
                x: 0.5
                y: 143.2
              }
              points {
                x: 0.6
                y: 141.7
              }
              points {
                x: 0.7
                y: 141
              }
              points {
                x: 0.75
                y: 140.8
              }
              points {
          

In [19]:
# Conversion of the hybrid propulsion system
# Create a pto system
switchboard_id = 1
rated_power = 1000
synch_mach = ElectricMachine(
    type_=TypeComponent.SYNCHRONOUS_MACHINE,
    power_type=TypePower.PTI_PTO,
    name="synchronous machine",
    rated_power=rated_power,
    rated_speed=1000,
    switchboard_id=switchboard_id,
)

# Create a rectifier instance
rectifier = ElectricComponent(
    type_=TypeComponent.RECTIFIER,
    power_type=TypePower.POWER_TRANSMISSION,
    name="rectifier",
    rated_power=rated_power,
    eff_curve=np.array([99.5]),
    switchboard_id=switchboard_id,
)

# Create a inverter instance
inverter = ElectricComponent(
    type_=TypeComponent.INVERTER,
    power_type=TypePower.POWER_TRANSMISSION,
    name="inverter",
    rated_power=rated_power,
    eff_curve=np.array([98.5]),
    switchboard_id=switchboard_id,
)

# Create a transformer instance
transformer = ElectricComponent(
    type_=TypeComponent.TRANSFORMER,
    power_type=TypePower.POWER_TRANSMISSION,
    name="transformer",
    rated_power=rated_power,
    eff_curve=np.array([99]),
    switchboard_id=switchboard_id,
)

pto = PTIPTO(
    name="PTI PTO",
    components=[synch_mach, rectifier, inverter, transformer],
    rated_power=1000,
    rated_speed=1000,
    switchboard_id=synch_mach.switchboard_id,
    shaft_line_id=1,
)

# Creat a new electric system with pto
all_electric_components = reduce(
    lambda acc, switchboard: [*acc, *switchboard.components],
    machinery_system.electric_system.switchboards.values(),
    [pto],
)
bus_tie_breakers = reduce(
    lambda acc, bus_tie_breaker: [*acc, bus_tie_breaker.switchboard_ids],
    machinery_system.electric_system.bus_tie_breakers,
    [],
)
electric_system_with_pto = ElectricPowerSystem(
    name="electric power system with PTO",
    power_plant_components=all_electric_components,
    bus_tie_connections=bus_tie_breakers,
)

# Create a new mechanical system with pto
all_mechanical_components = reduce(
    lambda acc, shaftline: [*acc, *shaftline.components],
    machinery_system.mechanical_system.shaft_line,
    [pto],
)

mechanical_system_with_pto = MechanicalPropulsionSystem(
    name="mechanical system with PTO",
    components_list=all_mechanical_components,
)

# Create a hybrid system
hybrid_prop_system_feems = HybridPropulsionSystem(
    name="Hybrid system",
    electric_system=electric_system_with_pto,
    mechanical_system=mechanical_system_with_pto,
)
hybrid_prop_system_proto = convert_hybrid_propulsion_system_to_protobuf(
    system_feems=hybrid_prop_system_feems
)
print(hybrid_prop_system_proto)

name: "Hybrid system"
propulsion_type: HYBRID
maximum_allowed_genset_load_percentage: 80
mechanical_system {
  shaft_lines {
    shaft_line_id: 1
    subsystems {
      electric_machine {
        name: "synchronous machine"
        rated_power_kw: 1000
        rated_speed_rpm: 1000
        efficiency {
          curve {
            x_label: "power load"
            y_label: "efficiency"
            curve {
              points {
                y: 1
              }
              points {
                x: 1
                y: 1
              }
            }
          }
        }
        order_from_switchboard_or_shaftline: 1
        uid: "56fae2e8-9421-4d5d-a5d1-4e7b09450d0a"
      }
      transformer {
        name: "transformer"
        rated_power_kw: 1000
        efficiency {
          curve {
            x_label: "power load"
            y_label: "efficiency"
            curve {
              points {
                y: 99
              }
              points {
                x:

In [20]:
# Save the systems to a file
electric_system_with_pto_proto = convert_electric_system_to_protobuf(
    electric_system=electric_system_with_pto
)
mechanical_system_with_pto_proto = convert_mechanical_system_to_protobuf(
    mechanical_propulsion_system=mechanical_system_with_pto
)
electric_propulsion_system_proto = convert_electric_system_to_protobuf_machinery_system(
    electric_system=electric_system, maximum_allowed_genset_load_percentage=80
)
files_to_save = {
    "electric_system_with_pto.mss": electric_system_with_pto_proto,
    "mechanical_system_with_pto.mss": mechanical_system_with_pto_proto,
    "electric_propulsion_system.mss": electric_propulsion_system_proto,
    "mechanical_propulsion_with_electric_system.mss": mechanical_propulsion_system_with_electric_system_proto,
    "hybrid_propulsion_system.mss": hybrid_prop_system_proto,
}
for file_name, proto_message in files_to_save.items():
    with open(os.path.join("tests", file_name), "wb") as f:
        f.write(proto_message.SerializeToString())

In [21]:
# Test with multi-fuel engines for different systems
import numpy as np
from feems.components_model.component_electric import (
    ElectricComponent,
    ElectricMachine,
    Genset,
)
from feems.components_model.component_mechanical import (
    EngineCycleType,
    EngineMultiFuel,
    FuelCharacteristics,
    MainEngineForMechanicalPropulsion,
    MechanicalPropulsionComponent,
    NOxCalculationMethod,
)
from feems.fuel import FuelOrigin, TypeFuel
from feems.system_model import ElectricPowerSystem, MechanicalPropulsionSystem
from feems.types_for_feems import Power_kW, Speed_rpm, TypeComponent, TypePower
from MachSysS.convert_to_protobuf import (
    convert_electric_system_to_protobuf,
    convert_mechanical_system_to_protobuf,
)

# Mechanical system with multi-fuel main engine
bsfc_for_main_engine = np.array(
    [
        [0.25, 151.2],
        [0.30, 149.5],
        [0.40, 146.3],
        [0.50, 143.2],
        [0.60, 141.7],
        [0.70, 141.0],
        [0.75, 140.8],
        [0.80, 140.9],
        [0.85, 141.0],
        [0.90, 141.4],
        [0.95, 141.9],
        [1.00, 142.8],
    ]
)
bspfc_for_main_engine = np.array(
    [
        [0.25, 1.6],
        [0.30, 1.4],
        [0.40, 1.1],
        [0.50, 0.9],
        [0.60, 0.8],
        [0.70, 0.8],
        [0.75, 0.7],
        [0.80, 0.7],
        [0.85, 0.7],
        [0.90, 0.6],
        [0.95, 0.6],
        [1.00, 0.6],
    ]
)
propeller = MechanicalPropulsionComponent(
    type_=TypeComponent.PROPELLER_LOAD,
    power_type=TypePower.POWER_CONSUMER,
    name="Propeller",
    rated_power=Power_kW(25000),
    rated_speed=Speed_rpm(60),
    shaft_line_id=1,
)
engine = EngineMultiFuel(
    type_=TypeComponent.MAIN_ENGINE,
    name="Main engine",
    rated_power=Power_kW(18000),
    rated_speed=Speed_rpm(60),
    multi_fuel_characteristics=[
        FuelCharacteristics(
            main_fuel_type=TypeFuel.NATURAL_GAS,
            main_fuel_origin=FuelOrigin.FOSSIL,
            pilot_fuel_type=TypeFuel.DIESEL,
            pilot_fuel_origin=FuelOrigin.FOSSIL,
            bsfc_curve=bsfc_for_main_engine,
            bspfc_curve=bspfc_for_main_engine,
            nox_calculation_method=NOxCalculationMethod.TIER_3,
            engine_cycle_type=EngineCycleType.OTTO,
        ),
        FuelCharacteristics(
            main_fuel_type=TypeFuel.VLSFO,
            main_fuel_origin=FuelOrigin.FOSSIL,
            bsfc_curve=bsfc_for_main_engine * 1.05,
            nox_calculation_method=NOxCalculationMethod.TIER_3,
        ),
    ],
    uid="cvjkrqwe23vc",
)
main_engine = MainEngineForMechanicalPropulsion(
    engine=engine,
    name="Main engine for mechanical propulsion",
    shaft_line_id=1,
    uid="cvjkrqwe23vc-shaftline",
)
# use the same propeller as before
mechanical_propulsion_system = MechanicalPropulsionSystem(
    name="Mechanical propulsion system with multi-fuel main engine",
    components_list=[main_engine, propeller],
)
mechanical_system_proto = convert_mechanical_system_to_protobuf(
    mechanical_propulsion_system=mechanical_propulsion_system
)
print(mechanical_system_proto)

# Electric system with multi-fuel gensets
number_gensets = 2
bsfc_for_aux_engine = np.array(
    [
        [0.25, 248.0],
        [0.50, 208.0],
        [0.75, 197.0],
        [1.00, 195.2],
    ]
)
bspfc_for_aux_engine = np.array(
    [
        [0.25, 2.5],
        [0.50, 1.8],
        [0.75, 1.5],
        [1.00, 1.4],
    ]
)
gensets = [
    Genset(
        name=f"Genset {i + 1}",
        aux_engine=EngineMultiFuel(
            type_=TypeComponent.AUXILIARY_ENGINE,
            name=f"Aux engine {i + 1}",
            rated_power=Power_kW(1290),
            rated_speed=Speed_rpm(900),
            multi_fuel_characteristics=[
                FuelCharacteristics(
                    main_fuel_type=TypeFuel.NATURAL_GAS,
                    main_fuel_origin=FuelOrigin.FOSSIL,
                    pilot_fuel_type=TypeFuel.DIESEL,
                    pilot_fuel_origin=FuelOrigin.FOSSIL,
                    bsfc_curve=bsfc_for_aux_engine,
                    bspfc_curve=bspfc_for_aux_engine,
                    nox_calculation_method=NOxCalculationMethod.TIER_3,
                    engine_cycle_type=EngineCycleType.OTTO,
                ),
                FuelCharacteristics(
                    main_fuel_type=TypeFuel.DIESEL,
                    main_fuel_origin=FuelOrigin.FOSSIL,
                    bsfc_curve=bsfc_for_aux_engine * 1.05,
                    nox_calculation_method=NOxCalculationMethod.TIER_3,
                    engine_cycle_type=EngineCycleType.DIESEL,
                ),
            ],
            uid=f"aux-engine-{i + 1}-uid",
        ),
        generator=ElectricMachine(
            type_=TypeComponent.GENERATOR,
            name=f"Generator {i + 1}",
            rated_power=Power_kW(1000),
            rated_speed=Speed_rpm(900),
            eff_curve=np.array(
                [
                    [0.0, 0.88],
                    [0.1, 0.89],
                    [0.2, 0.90],
                    [0.3, 0.91],
                    [0.4, 0.92],
                    [0.5, 0.93],
                    [0.6, 0.93],
                    [0.7, 0.94],
                    [0.8, 0.94],
                    [0.9, 0.95],
                    [1.0, 0.95],
                ]
            ),
            power_type=TypePower.POWER_SOURCE,
            switchboard_id=int(i / 2) + 1,
            uid=f"generator-{i + 1}-uid",
        ),
        uid=f"genset-{i + 1}-uid",
    )
    for i in range(number_gensets)
]
other_load = ElectricComponent(
    type_=TypeComponent.OTHER_LOAD,
    name="Other load",
    rated_power=Power_kW(4000),
    power_type=TypePower.POWER_CONSUMER,
    switchboard_id=1,
    uid="other-load-uid",
)
electric_power_system = ElectricPowerSystem(
    name="Electric power system with multi-fuel gensets",
    power_plant_components=[*gensets, other_load],
    bus_tie_connections=[(1, 1)],
)
electric_system_proto = convert_electric_system_to_protobuf(electric_system=electric_power_system)
print(electric_system_proto)

shaft_lines {
  shaft_line_id: 1
  subsystems {
    power_type: POWER_SOURCE
    component_type: MAIN_ENGINE
    name: "Main engine for mechanical propulsion"
    rated_power_kw: 18000
    rated_speed_rpm: 60
    uid: "cvjkrqwe23vc-shaftline"
    multi_fuel_engine {
      name: "Main engine"
      rated_power_kw: 18000
      rated_speed_rpm: 60
      fuel_modes {
        main_fuel {
          fuel_type: NATURAL_GAS
          fuel_origin: FOSSIL
        }
        main_bsfc {
          curve {
            x_label: "power load"
            y_label: "bsfc"
            curve {
              points {
                x: 0.25
                y: 151.2
              }
              points {
                x: 0.3
                y: 149.5
              }
              points {
                x: 0.4
                y: 146.3
              }
              points {
                x: 0.5
                y: 143.2
              }
              points {
                x: 0.6
                y: 141.7
 